In [1]:
from abc import ABC,abstractmethod
import numpy as np

In [2]:
class Layer(ABC):

    @property
    @abstractmethod
    def output(self):
        ...
  
    @abstractmethod
    def __call__ (self,input_tensor:np.ndarray)->np.ndarray:
        ...

    @abstractmethod
    def build (self,input_tensor:np.ndarray):
        ...
  
    @abstractmethod
    def update (self, learning_rate:float):
        ...
  

In [3]:
class Dense(Layer):
    SGD = "sgd"
    HE_NORM = "he_normal"
    GLORAT_NORM = "glorat_normal"

    def __init__ (self,units:int,weight_kernel="he_normal"):
        """
        attributes:
        units -> int
        weight_kernel = "he_normal","glorat_normal"
        """
        self._units = units
        self._input_units = None
        self._weights:np.ndarray = None
        self._bias:np.ndarray = None
        self._d_w:np.ndarray = None
        self._d_b:np.ndarray = None
        self._output = None
        self._weight_kernel:str = weight_kernel
        self._optimizer:str = Dense.SGD

    @property
    def grad_weights(self):
        return self._d_w
  
    @grad_weights.setter
    def grad_weights(self,gradients:np.ndarray):
        self._d_w = gradients
  
    @property
    def grad_bias(self):
        return self._d_b

    @grad_bias.setter
    def grad_bias(self,gradients:np.ndarray):
        self._d_b = gradients

    @property
    def weights(self):
        return self._weights
  
    @property
    def bias(self):
        return self._bias

    @property
    def output(self):
        return self._output

    @property
    def optimizer(self):
        return self._optimizer

    @optimizer.setter
    def optimizer(self,optimizer:str):
        self._optimizer = optimizer

  
    def weight_kernel(self)->float:
        if self._weight_kernel == Dense.HE_NORM:
            std = np.sqrt(2.0/(self._input_units))
        elif self._weight_kernel == Dense.GLORAT_NORM:
            std = np.sqrt(2.0/(self._input_units + self._units))
        return std


    def build(self,input_tensor:np.ndarray):
        self._input_units = input_tensor.shape[0]
        self._weights = np.random.randn(self._units,self._input_units)*self.weight_kernel()
        self._bias = np.zeros((self._units,1))

    def __call__ (self,input_tensor:np.ndarray)->np.ndarray:
        if self._weights is None:
            self.build(input_tensor=input_tensor)

        self._output = np.dot(self._weights,input_tensor) + self._bias
        return self._output

    def update (self, learning_rate:float):
        if self._optimizer == Dense.SGD:
            self._sgd(learning_rate=learning_rate)

    def _sgd(self, learning_rate:float):
        self._weights = self._weights + learning_rate*self._d_w
        self._bias = self._bias + learning_rate*self._d_b
